In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import requests

In [2]:
overwatch_api = 'https://api.overwatchleague.com/stats/players'
res = requests.get(overwatch_api)

In [5]:
watchdata = res.json()
with open('./overstat.json', 'w', encoding = 'utf-8') as make_file:
    json.dumps(watchdata, indent = '\t')

In [6]:
df = pd.DataFrame(watchdata['data'])
df.head()

,playerId,teamId,role,name,team,eliminations_avg_per_10m,deaths_avg_per_10m,hero_damage_avg_per_10m,healing_avg_per_10m,ultimates_earned_avg_per_10m,final_blows_avg_per_10m,time_played_total
0,5197,7693,offense,Ado,HZS,11.993982,5.611274,5234.956801,6207.457948,4.196975,3.765339,65332.762069
1,8675,7693,offense,Adora,HZS,16.002129,3.876235,5390.964430,5678.864360,4.410253,4.134037,65166.328327
2,4142,7695,offense,Agilities,TOR,15.930212,5.525910,6591.302371,6535.752019,4.740087,4.815526,95441.290463
3,5863,7697,support,AimGod,WAS,13.414336,4.687159,6721.795662,9486.871283,4.724139,3.651732,64900.715138
4,3985,4523,offense,aKm,DAL,17.296143,5.178533,9600.623636,158.646056,4.575825,6.050875,75658.485169


크롤링이 생각보다 아주 단순하다

In [ ]:
#역할군별 10분당 처치비율
avg_eli_per_10 = df.groupby('role')['eliminations_avg_per_10m'].mean()
print(avg_eli_per_10)

role
offense    16.651697
support    12.737868
tank       16.503044
Name: eliminations_avg_per_10m, dtype: float64


In [ ]:
#리그 전체 수준의 목처 비율
df['eli_per_death'] = (df['eliminations_avg_per_10m'] / df['deaths_avg_per_10m'])
df['eli_per_death'].mean()

3.320161794038524

In [ ]:
#기술통계량으로 그 데이터계열의 구간을 알아볼 수 있음
df['eli_per_death'].describe()

count    126.000000
mean       3.320162
std        0.916125
min        0.891304
25%        2.677959
50%        3.184778
75%        3.858503
max        6.084862
Name: eli_per_death, dtype: float64

In [ ]:
# eli_per_death_edge 어느 정도해야 리그 평균인지 알 수 있게 구간 설정
eli_per_death_edges = [1.9,3.164157,3.783626, 6.1]

In [ ]:
eli_per_death_names = ['low-performer', 'avg-performer', 'super-performer']

In [ ]:
#위의 기술통계량을 바탕으로 상위, 하위 레빌 나눔, 그리고 pd.cut을 이용해서 구간별로 잘라서 새로운 열로 만들어 player 기준으로 붙여줌
df['eli_per_death_level'] = pd.cut(df['eli_per_death'], eli_per_death_edges, labels = eli_per_death_names)
df.head(3)

,playerId,teamId,role,name,team,eliminations_avg_per_10m,deaths_avg_per_10m,hero_damage_avg_per_10m,healing_avg_per_10m,ultimates_earned_avg_per_10m,final_blows_avg_per_10m,time_played_total,eli_per_death,eli_per_death_level
0,5197,7693,offense,Ado,HZS,11.993982,5.611274,5234.956801,6207.457948,4.196975,3.765339,65332.762069,2.137480,low-performer
1,8675,7693,offense,Adora,HZS,16.002129,3.876235,5390.964430,5678.864360,4.410253,4.134037,65166.328327,4.128266,super-performer
2,4142,7695,offense,Agilities,TOR,15.930212,5.525910,6591.302371,6535.752019,4.740087,4.815526,95441.290463,2.882821,low-performer


In [ ]:
#상위 20명의 목처 비율은 뭔가?
top_twens = df.nlargest(20, 'eli_per_death')
top_twens['eli_per_death'].mean()

4.836548404637718

In [ ]:
#딜러 군 상위 20명의 목처 비율은 뭔가?
dealer = df.loc[df['role'] == 'offense'].copy()
dealer.head()

,playerId,teamId,role,name,team,eliminations_avg_per_10m,deaths_avg_per_10m,hero_damage_avg_per_10m,healing_avg_per_10m,ultimates_earned_avg_per_10m,final_blows_avg_per_10m,time_played_total,eli_per_death,eli_per_death_level
86,8236,4403,offense,Nenne,NYE,19.365667,3.759414,9397.008782,110.601006,4.348077,6.869962,89694.820843,5.151246,super-performer
51,8750,4403,offense,Haksal,NYE,19.341104,4.282244,6198.426230,7396.956136,4.975924,5.462046,109848.948995,4.516582,super-performer
49,8713,7693,offense,GodsB,HZS,17.820443,3.996136,9683.685510,198.616918,4.516307,6.603111,98044.700710,4.459418,super-performer
105,4099,4403,offense,SAEBYEOLBE,NYE,17.099836,3.964827,5885.124875,818.279686,4.208068,5.910754,24666.903576,4.312883,super-performer
96,4662,4409,offense,Profit,SEO,18.610000,4.384087,8591.855489,938.785799,4.618088,6.359334,87178.935509,4.244898,super-performer


In [ ]:
dealer_twen = dealer.nlargest(20, 'eli_per_death')
dealer_twen['eli_per_death'].mean()

4.071400483713899

탱커군이 더 높을 수도 있네

In [ ]:
df_super = df.loc[df['eli_per_death_level'] == 'super-performer'].copy()
df_super.head()

,playerId,teamId,role,name,team,eliminations_avg_per_10m,deaths_avg_per_10m,hero_damage_avg_per_10m,healing_avg_per_10m,ultimates_earned_avg_per_10m,final_blows_avg_per_10m,time_played_total,eli_per_death,eli_per_death_level
124,5809,4408,tank,Void,SHD,19.343389,3.178936,6457.220391,250.965632,2.048810,5.555847,82291.682828,6.084862,super-performer
62,8748,7697,tank,JJANU,WAS,20.056908,3.381134,6203.063756,215.480218,2.321375,5.551115,107005.528668,5.932007,super-performer
83,8242,4409,tank,Michelle,SEO,18.928176,3.239774,6474.671283,235.795296,2.634018,5.737667,88154.295335,5.842437,super-performer
24,5831,4404,tank,Choihyobin,SFS,19.391551,3.472553,6035.142896,302.338209,2.882165,5.957548,111790.956037,5.584235,super-performer
6,4841,4403,support,Anamo,NYE,14.165021,2.745207,3948.127250,6563.244540,3.567532,1.898150,97041.863429,5.159910,super-performer


In [ ]:
df_super.groupby(['team'])['eli_per_death_level'].count()

team
ATL    1
CDH    1
FLA    1
GLA    1
GZC    2
HOU    2
HZS    4
NYE    6
PHI    2
SEO    4
SFS    3
SHD    3
TOR    1
WAS    2
Name: eli_per_death_level, dtype: int64

In [ ]:
#북미와 아시아 플레이오프 3라운드 팀만 골라냄, 마지막에 : 또 까먹음
df_usa = df.loc[df['team'].isin(['PHI','GLA', 'FLA', 'VAL', 'SFS', 'WAS', 'PAR', 'ATL']), :].copy()
df_asia = df.loc[df['team'].isin(['SHD', 'NYE', 'GZC', 'SEO']), :].copy()

In [ ]:
#출전 시간을 시간 단위로 바꾸기
df_usa['time_played_total'] = df_usa['time_played_total'].apply(lambda x: x / 3600)
df_asia['time_played_total'] = df_usa['time_played_total'].apply(lambda x: x / 3600)

In [ ]:
df_usa.head()

,playerId,teamId,role,name,team,eliminations_avg_per_10m,deaths_avg_per_10m,hero_damage_avg_per_10m,healing_avg_per_10m,ultimates_earned_avg_per_10m,final_blows_avg_per_10m,time_played_total,eli_per_death,eli_per_death_level
62,8748,7697,tank,JJANU,WAS,20.056908,3.381134,6203.063756,215.480218,2.321375,5.551115,29.723758,5.932007,super-performer
24,5831,4404,tank,Choihyobin,SFS,19.391551,3.472553,6035.142896,302.338209,2.882165,5.957548,31.053043,5.584235,super-performer
42,4659,4524,tank,Fury,PHI,18.857755,3.903886,6670.623658,231.368301,3.619366,6.213134,25.188576,4.830508,super-performer
40,9591,7698,tank,frd,ATL,18.613398,3.905223,5983.492229,477.419815,2.962583,6.224416,11.138930,4.766284,super-performer
114,4637,4406,tank,SPACE,GLA,16.937870,3.754919,6297.654849,349.497358,4.028947,5.421303,22.503813,4.510848,super-performer


In [ ]:
df_usa_graph = df_usa.groupby(['team'])[['hero_damage_avg_per_10m']].sum()
df_asia_graph = df_usa.groupby(['team'])[['hero_damage_avg_per_10m']].sum()

In [ ]:
df_usa_graph.reset_index(inplace=True)

In [ ]:
df_asia_graph.reset_index(inplace=True)

In [ ]:
df_asia_graph

,team,hero_damage_avg_per_10m
0,ATL,36621.304587
1,FLA,39591.619961
2,GLA,43123.989634
3,PAR,26446.296302
4,PHI,53969.524367
5,SFS,52789.861557
6,VAL,28822.329989
7,WAS,38920.964962


위의 데이터를 엑셀 파일에 차트와 표로 만들고 저장하기

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 2.9MB/s 


In [ ]:
#경로설정 및 엑셀파일 쓰기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#경로설정 필요
writer = pd.ExcelWriter('/content/drive/My Drive/goorum/overwatch_refined.xlsx', engine='xlsxwriter')

In [ ]:
# 데이터프레임의 객체를 XlsxWriter Excel 객체로 변환 
df_usa_graph.to_excel(writer, sheet_name='Sheet1', index=False)

# xlsxwriter workbook and worksheet 객체 호출
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

In [ ]:
#엑셀의 셀조정
chart = workbook.add_chart({'type': 'column'})
chart.add_series({
    'values':     '=Sheet1!$B$2:$B$9',
    'categories': '=Sheet1!$A$2:$A$9',
    'gap':        100, # column 사이의 간격 
})

In [ ]:
#헤더포맷
header_format = workbook.add_format({
    'size' : 10,
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'align' : 'center',
    'fg_color': '#D7E4BC',
    'border': 1})

In [ ]:
for col_num, value in enumerate(df_usa_graph.columns.values):  #값을 루프문으로 삽입
    worksheet.write(0, col_num, value, header_format)

# Cell의 포맷팅 설정 
format1 = workbook.add_format({'num_format': '#,##0.00'})
format2 = workbook.add_format({'num_format':  '#,##0.00'})

# B:B 전체, Cell 크기 20, format 설정은 위의 format1
worksheet.set_column('A:A', 20, format1)

# C:C 전체, Cell 크기 기본값 , format 설정은 위의 format2
worksheet.set_column('B:B', 15, format2)

worksheet.write(0,2, "(unit:damage)")
writer.save()